In [113]:
import geopandas as gpd 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [115]:
# load the data
dataset = gpd.read_file('./data.gpkg')
print('dataset size: ', dataset.shape[0])
print('dataset columns: ', dataset.columns.values)

dataset size:  1456
dataset columns:  ['TxID' 'StrNr' 'Strak' 'BdlNr' 'Bandel' 'Forbind_1' 'PlSignFr' 'PlNamnFr'
 'PlSignTi' 'PlNamnTi' 'AntalSpar' 'Elektrif' 'Infraforv' 'UHDist' 'TrSys'
 'TComr' 'Status' 'BdlSeq' 'geometry']


In [116]:
# load the dataset containg the information if a passenger can board a train
stations = gpd.read_file('./stations.gpkg')
print('dataset size: ', stations.shape[0])
print('dataset columns: ', stations.columns.values)
# the column ResUtbyte is the one we are interested in below:

dataset size:  1643
dataset columns:  ['OBJECTID' 'TplKod' 'Sign' 'SignTxt' 'PlNamn' 'PlTyp' 'PltypBesk'
 'Tillh_dpd' 'Tillh_dp_lp' 'PlStatus' 'ResUtbyte' 'Lastpl' 'Hamn'
 'Kombiterm' 'Northing' 'Easting' 'Latitud' 'Longitud' 'Moh' 'Strak'
 'Bandel' 'Kmtalfr' 'Kmtalti' 'Region' 'TL' 'UH' 'Dignitet' 'Skapad'
 'geometry']


In [117]:
# get all the stations where a passenger can board a train
p_stations = stations[stations.ResUtbyte == 'j']
print('size:', p_stations.shape)
p_stations

size: (860, 29)


,OBJECTID,TplKod,Sign,SignTxt,PlNamn,PlTyp,PltypBesk,Tillh_dpd,Tillh_dp_lp,PlStatus,...,Strak,Bandel,Kmtalfr,Kmtalti,Region,TL,UH,Dignitet,Skapad,geometry
33,786741,None,xTib,Tib,Triabo,xhp,Hållplats på NJDB-nätet. Får ej anv på TRV spår,,,Öppen,...,,001,0+0,0+0,Region Syd,,Övr,,20231210,POINT (532324.333 6343895.667)
34,786742,None,xTob,Tob,Totebo,stn,Station - Får ej användas på TRV spår,,,Öppen,...,,001,34+598,34+598,Region Syd,,Övr,,20231210,POINT (570520.223 6387909.758)
37,786745,None,xGyt,Gyt,Gyttorp,stn,Station - Får ej användas på TRV spår,,,Öppen,...,,001,22+612,22+612,Region Öst,,Övr,,20231210,POINT (498371.441 6595914.771)
38,786746,None,xÅtp,Åtp,Åtorp,stn,Station - Får ej användas på TRV spår,,,Öppen,...,,001,0+0,0+0,Region Väst,,Övr,,20231210,POINT (306551.724 6484437.030)
39,786747,None,xFåt,Fåt,Fårhult,xhp,Hållplats på NJDB-nätet. Får ej anv på TRV spår,,,Öppen,...,,001,13+351,13+351,Region Syd,,Övr,,20231210,POINT (584544.706 6400395.706)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,788346,SE3652,Sol,Sol,Sollentuna,dp,Driftplats,,,Öppen,...,05,433,13+200,13+200,Region Stockholm,Cst,Öst/Stockholm,,20231210,POINT (667288.718 6591411.297)
1639,788347,None,Nvk,Nvk,Norrviken,hp,Hållplats,,,Öppen*,...,05,433,17+25,17+25,Region Stockholm,Cst,Öst/Stockholm,,20231210,POINT (665731.996 6594848.186)
1640,788348,SE3233,R,R,Rotebro,dp,Driftplats,,,Öppen,...,05,433,18+898,18+898,Region Stockholm,Cst,Öst/Stockholm,,20231210,POINT (665137.731 6596606.357)
1641,788349,SE4348,Upv,Upv,Upplands Väsby,dp,Driftplats,,,Öppen,...,05,433,24+266,24+266,Region Stockholm,Cst,Öst/Stockholm,,20231210,POINT (664004.840 6601576.626)


In [ ]:
dataset.head()

In [ ]:
df = dataset[['PlSignFr', 'PlSignTi']]
df

In [ ]:
df = pd.concat([df, pd.DataFrame(data=df[['PlSignTi', 'PlSignFr']].values, columns=['PlSignFr', 'PlSignTi'])])
df

In [ ]:
adj_matrix = pd.crosstab(df.PlSignFr, df.PlSignTi)
adj_matrix

In [ ]:
adj_matrix_numpy = adj_matrix.to_numpy()

n = 1
result = np.linalg.matrix_power(adj_matrix_numpy , n)
print("grannmatrisen upphöjt till", n, "är:")
print(result)

# Räkna antalet icke-noll element i varje rad
non_zero_counts = np.count_nonzero(result, axis=1)

# Hämta indexen för de 10 raderna med flest icke-noll element
top_10_indices = np.argsort(non_zero_counts)[::-1][:10]

print("Top 10 rader med flest icke-noll element är:")
for index in top_10_indices:
    print("Rad", adj_matrix.columns[index], ":", result[index])
    print("Antal icke-noll element:", non_zero_counts[index])